In [ ]:
!pip install onnxruntime==1.9.0 -q
!pip install -U tf2onnx==1.9.2 -q

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tf2onnx
import onnxruntime
from sklearn.metrics import classification_report as Report
from onnxruntime.quantization import quantize_dynamic

# Mnist

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11501568/11490434 [==============================] - 0s 0us/step
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
     
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        
        layers.BatchNormalization(),
        layers.Flatten(),
        
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 1, 1, 64)          0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 1, 1, 64)         

In [ ]:
batch_size = 256
epochs = 10

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/10
211/211 [==============================] - 35s 16ms/step - loss: 0.2751 - accuracy: 0.9275 - val_loss: 0.7320 - val_accuracy: 0.9578
Epoch 2/10
211/211 [==============================] - 3s 14ms/step - loss: 0.0764 - accuracy: 0.9799 - val_loss: 0.1043 - val_accuracy: 0.9788
Epoch 3/10
211/211 [==============================] - 3s 14ms/step - loss: 0.0513 - accuracy: 0.9863 - val_loss: 0.0595 - val_accuracy: 0.9822
Epoch 4/10
211/211 [==============================] - 3s 15ms/step - loss: 0.0389 - accuracy: 0.9893 - val_loss: 0.0583 - val_accuracy: 0.9827
Epoch 5/10
211/211 [==============================] - 3s 14ms/step - loss: 0.0296 - accuracy: 0.9918 - val_loss: 0.0548 - val_accuracy: 0.9837
Epoch 6/10
211/211 [==============================] - 3s 14ms/step - loss: 0.0237 - accuracy: 0.9934 - val_loss: 0.0442 - val_accuracy: 0.9862
Epoch 7/10
211/211 [==============================] - 3s 14ms/step - loss: 0.0187 - accuracy: 0.9953 - val_loss: 0.0438 - val_accuracy: 0.986

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.054995469748973846
Test accuracy: 0.9811000227928162


# Convert to ONNX

In [ ]:
# Конвертирование
onnx_model, _ = tf2onnx.convert.from_keras(model=model)

In [ ]:
onnx_model

In [ ]:
tf2onnx.onnx.save_model(onnx_model, 'model.onnx')

# Onnx

In [ ]:
sess = onnxruntime.InferenceSession('model.onnx')

In [ ]:
inputs = sess.get_inputs()
for inp in inputs:
  print(inp.name)

input_1


In [ ]:
def OnnxTest(x_test, y_test, sess):
  input_name = sess.get_inputs()[0].name
  output_name = sess.get_outputs()[0].name
  outp = sess.run([output_name], {input_name: x_test})

  class_pred = []
  for prob in outp[0]:
    class_pred.append(np.argmax(prob))

  class_real = []
  for real in y_test:
    class_real.append(np.argmax(real))

  print(Report(class_real, class_pred))

In [ ]:
OnnxTest(x_test, y_test, sess)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       1.00      1.00      1.00      1135
           2       0.99      0.99      0.99      1032
           3       0.99      0.99      0.99      1010
           4       1.00      0.99      1.00       982
           5       0.99      0.99      0.99       892
           6       0.99      0.99      0.99       958
           7       0.98      1.00      0.99      1028
           8       0.99      0.99      0.99       974
           9       0.99      0.99      0.99      1009

    accuracy                           0.99     10000
   macro avg       0.99      0.99      0.99     10000
weighted avg       0.99      0.99      0.99     10000



# Квантование

In [ ]:
quantize_dynamic('/content/model.onnx', '/content/model_q.onnx')

In [ ]:
sess_q = onnxruntime.InferenceSession('model_q.onnx')

In [ ]:
OnnxTest(x_test, y_test, sess_q)

              precision    recall  f1-score   support

           0       0.99      0.98      0.98       980
           1       0.99      1.00      0.99      1135
           2       0.97      0.99      0.98      1032
           3       0.99      0.99      0.99      1010
           4       0.99      0.98      0.98       982
           5       0.99      0.96      0.97       892
           6       0.99      0.98      0.99       958
           7       0.99      0.96      0.97      1028
           8       0.98      0.99      0.98       974
           9       0.93      0.99      0.96      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000

